<a href="https://colab.research.google.com/github/Erifai/Bi_Project/blob/main/Projet_BI_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Préparation de l'environnement :**

In [25]:
#!pip install -U spacy 
#!pip install spacy-transformers
#!pip install spacy_entity_linker
#!python3 -m spacy_entity_linker "download_knowledge_base"
#!python -m spacy download en_core_web_sm
#!pip install -U sentence-transformers
#!sudo apt-get install metis
#!pip install Cluster_Ensembles

#à executer une seule fois et de redémarrer le runtime avec GPU 

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from spacy.lang.en.stop_words import STOP_WORDS
import string
import re
import operator
from functools import reduce


# **Chargement et nettoyage de données :**

In [2]:
#df = pd.read_fwf('/content/DBLP_Subset.txt')
#df=pd.read_table('/content/DBLP_Subset.txt',header=None)
#df = pd.read_csv("/content/DBLP_Subset.txt",sep="\n")
reading_file = open("/content/DBLP_Subset.txt", "r")

new_file_content = ""
count = 0
for line in reading_file:

  stripped_line = line.strip()

  new_line = stripped_line.replace(';',',')
  if "#!" in new_line :
    count=0
  new_line = new_line.replace('#c',';').replace('#*','\n;').replace('#c',';').replace('#t',';').replace('#index',';').replace('#!',';').replace('#@',';')
  if "#%" in new_line and count==0 :
    #print(new_line)
    new_line = new_line.replace('#%',';')
    #print(new_line)

    count =1


  new_file_content += new_line


reading_file.close()
df = pd.DataFrame([x.split(';') for x in new_file_content.split('\n')])
# .replace('#c',';').replace('#*',';').replace('#c%',';').replace('#t',';').replace('#index',';').replace('#!',';')
#   df = pd.DataFrame([x.split(';') for x in lines.split('\n')])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37964 entries, 0 to 37963
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       37964 non-null  object
 1   1       37963 non-null  object
 2   2       37963 non-null  object
 3   3       37963 non-null  object
 4   4       37963 non-null  object
 5   5       37963 non-null  object
 6   6       20875 non-null  object
 7   7       17852 non-null  object
dtypes: object(8)
memory usage: 2.3+ MB


In [4]:
df.rename(columns={0:"supprimer",1:"Titre",2:"Auteurs",3:"Année",4:"Publication",5:"IndexId",6:"IdRefrences",7:"Abstract"},inplace= True)
df.drop(columns=['supprimer'],inplace= True)
df.dropna(subset=['Abstract'],inplace= True)
df.info()

In [35]:
#df.to_csv("df.csv", sep=';', encoding='utf-8')

In [36]:
#df = pd.read_csv("/content/df.csv", sep = ";",header=None, names=["Titre","Auteurs","Année","Publication","IndexId","IdRefrences","Abstract"])

In [37]:
#df.info()

In [8]:
df = df[0:100]

In [9]:
#punctuations
punctuations = string.punctuation
#stopwords
stop_words = list(STOP_WORDS)
def clean_text(
  string : str,
  ponctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~',
  stop_words = stop_words)->str : 

  #URLs
  string = re.sub(r"https?://\S+|www\.\S+",'',string)
  #html 
  string = re.sub(r'<.*?>','',string)
  #Numbers
  string = re.sub(r'[0-9]+','',string) 

  #Remove punctuations
  for x in string.lower():
    if x in punctuations:
      string = string.replace(x,"")
  #To lower
  string = string.lower()
  #Remove stop words
  string = ' '.join([word for word in string.split() if word not in stop_words])
  #Cleaning the whitespaces
  string = re.sub(r'\s', ' ', string).strip()
  return string
  


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
abstract = df.Abstract.to_list()
abstracts_cleaned = [clean_text(x) for x in abstract]
len(abstracts_cleaned)

# **Transformation de données :**

## **Vectorisation (Document - mot )**

In [13]:
vectorizer = CountVectorizer()
DocTerm_abstract = vectorizer.fit_transform(abstracts_cleaned)
print('type of DocTerm_abstract : ', type(DocTerm_abstract))
print("Documents / Mots  : ",DocTerm_abstract.shape)
print(DocTerm_abstract)

DocTerm_abstract = DocTerm_abstract.toarray()
print("******************************************")
print('DocTerm_abstract', type(DocTerm_abstract))


type of DocTerm_abstract :  <class 'scipy.sparse.csr.csr_matrix'>
Documents / Mots  :  (100, 2657)
  (0, 312)	2
  (0, 2041)	3
  (0, 143)	2
  (0, 1149)	1
  (0, 1437)	1
  (0, 1472)	1
  (0, 2125)	1
  (0, 691)	1
  (0, 391)	1
  (0, 1921)	1
  (0, 18)	1
  (0, 921)	1
  (0, 1474)	1
  (0, 2625)	1
  (0, 2276)	1
  (0, 1130)	1
  (0, 66)	1
  (0, 1826)	1
  (0, 1837)	1
  (0, 2193)	1
  (0, 671)	1
  (0, 686)	1
  (0, 845)	1
  (0, 2246)	1
  (0, 223)	1
  :	:
  (99, 2025)	1
  (99, 1688)	1
  (99, 1276)	1
  (99, 1509)	1
  (99, 1785)	1
  (99, 959)	1
  (99, 1070)	2
  (99, 405)	1
  (99, 1380)	1
  (99, 889)	2
  (99, 1949)	1
  (99, 2494)	1
  (99, 609)	1
  (99, 2207)	1
  (99, 1657)	3
  (99, 696)	1
  (99, 2493)	1
  (99, 2594)	1
  (99, 946)	1
  (99, 254)	1
  (99, 997)	1
  (99, 1504)	1
  (99, 1534)	1
  (99, 1092)	1
  (99, 1758)	1
******************************************
DocTerm_abstract <class 'numpy.ndarray'>


In [14]:
#Get Words List
print(vectorizer.get_feature_names())


['ability', 'able', 'abstract', 'abstraction', 'acceleration', 'acceptable', 'access', 'accesses', 'accommodate', 'accompanying', 'accomplish', 'account', 'accountand', 'accuracy', 'accuracybut', 'accurate', 'accurately', 'accuratelyand', 'achievable', 'achieve', 'achieved', 'achieves', 'achieving', 'achilles', 'act', 'activation', 'activity', 'activitybased', 'acts', 'actual', 'adapting', 'adaption', 'adaptive', 'added', 'adding', 'additional', 'additionour', 'address', 'addressed', 'addresses', 'addressing', 'adequately', 'adhoc', 'adjacenciesshould', 'adjacency', 'adjacencybased', 'adjust', 'adopting', 'advanced', 'advantage', 'advantages', 'advisor', 'affecting', 'affordable', 'againefficiently', 'aggregating', 'aggressor', 'ahb', 'ai', 'aided', 'algebraic', 'algorithm', 'algorithmas', 'algorithmcalled', 'algorithmknown', 'algorithmnamed', 'algorithms', 'algorithmscalled', 'algorithmsdogmam', 'algorithmsnamed', 'alignment', 'allcritical', 'allocate', 'allotted', 'allour', 'allow', 

## **Exctraction de "Linked entities"**

In [15]:
import spacy
import en_core_web_sm
from functools import reduce
nlp = spacy.load('en_core_web_sm')
# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker",last=True)

In [16]:
listEntity =[]
for a,abs in enumerate(abstracts_cleaned):
  doc = nlp(str(abs))
  listInter=[]
  #return all the entities in the whole document
  all_linked_entities = doc._.linkedEntities
  #itereate over sentences and print linked entities
  for sent in doc.sents:
    #print(sent._.linkedEntities)
    for i in range(len(sent._.linkedEntities)):
      entity = sent._.linkedEntities[i].get_id()
      listInter.append(entity)
  listEntity.append(listInter)


In [17]:
listEntityAll = reduce(operator.concat,listEntity)
print("listEntityAll" , len(listEntityAll))
listEntityAllunique = np.unique(np.asarray(listEntityAll))
print("listEntityAllunique" , len(listEntityAllunique))


listEntityAll 1682
listEntityAllunique 566


## **BOW**

In [19]:
dictindex = dict(zip(listEntityAllunique,np.arange(len(listEntityAllunique))))
entityEmbedding = np.zeros((len(abstract),len(listEntityAllunique)))
for a in range(len(abstract)):
  lista = listEntity[a]
  indexEn = values = np.asanyarray(list(map(dictindex.get,lista)))
  entityEmbedding[a,indexEn]=1

## **BERT**

In [20]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embedding = model.encode(abstract)

In [21]:
sentence_embedding

array([[-0.90498275,  0.20419133,  0.54003567, ...,  0.1595102 ,
        -0.02890011,  0.21780182],
       [-0.84162533,  0.37580818,  0.31556702, ..., -0.15957555,
        -0.1251584 ,  0.36724833],
       [-0.0920645 ,  0.60817575,  0.5032281 , ..., -0.46368083,
        -1.1678922 ,  0.628837  ],
       ...,
       [-0.8522707 ,  0.6357087 ,  0.32293302, ..., -0.90342045,
        -0.16440001,  0.16181709],
       [-0.15328439,  0.30836326,  1.0114954 , ..., -0.23881237,
        -0.5476271 ,  0.27469596],
       [-0.00146912,  0.23846017,  0.8166513 , ..., -0.25199023,
        -1.2100103 ,  0.4056557 ]], dtype=float32)

# **Clustering**

In [22]:
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

K=3
#labels_all = np.asarray(df['label']).astype(int)

kmeans = KMeans(n_clusters=K, random_state=0,n_init=5).fit(DocTerm_abstract)
y_pred1 = kmeans.labels_
print(y_pred1)
#print('Bow representation clustering  accuracy : ',accuracy_score(labels_all,y_pred1))

In [23]:
kmeans = KMeans(n_clusters=K, random_state=0,n_init=5).fit(entityEmbedding)
#kmeans = KMeans(n_clusters=K, random_state=0,n_init=5).fit(mat)

y_pred2 = kmeans.labels_
print(y_pred2)
#print('Entity  representation clustering  accuracy : ',accuracy_score(labels_all,y_pred2))

In [24]:
kmeans = KMeans(n_clusters=K, random_state=0,n_init=5).fit(sentence_embedding)
y_pred3 = kmeans.labels_
print(y_pred3)
#print('Sentence  representation clustering  accuracy : ',accuracy_score(labels_all,y_pred3))

In [25]:
import numpy as np
import Cluster_Ensembles as CE
import functools
from functools import reduce
clustering_1 = y_pred1
clustering_2 = y_pred2
clustering_3 = y_pred3
cluster_runs = np.array([clustering_1,clustering_2,clustering_3])
print("cluster_runs" , cluster_runs)
#consensus_clustering_labels = CE.cluster_ensembles(cluster_runs,verbose = True,N_clusters_max = 3)
#print("consensus_clustering_labels",consensus_clustering_labels)

cluster_runs [[2 0 1 1 1 2 2 2 2 2 0 2 2 2 0 2 1 2 0 1 2 1 1 2 2 2 1 2 1 1 0 2 2 2 2 2
  2 1 2 2 2 2 2 0 0 2 1 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0
  1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 2 1 2 2 2 1 1]
 [2 2 0 0 1 1 1 2 1 1 1 1 1 2 1 0 1 1 0 2 0 1 2 1 2 1 0 2 1 1 2 2 1 1 1 0
  1 1 1 0 1 1 2 2 1 0 2 2 2 2 2 2 1 1 1 2 2 1 1 1 2 0 1 1 0 1 1 1 1 1 1 1
  1 0 1 1 1 0 2 1 1 1 1 0 0 1 0 1 1 1 0 1 1 0 1 2 1 1 1 1]
 [1 1 1 1 1 1 1 1 2 1 1 0 1 0 1 0 2 2 2 1 1 1 1 0 0 0 1 0 1 0 1 0 2 1 2 1
  1 2 1 0 2 0 0 0 1 0 1 0 0 1 1 1 1 1 1 0 0 0 2 2 0 0 0 1 1 1 1 1 1 2 2 1
  1 0 2 0 2 0 1 1 2 2 2 0 1 1 0 2 1 0 1 0 0 1 0 1 2 1 1 1]]


NameError: ignored

In [ ]:
#from sklearn.metrics import accuracy_score
#print('Sentence  representation clustering  accuracy : ',accuracy_score(labels_all  ,consensus_clustering_labels ))

Sentence  representation clustering  accuracy :  0.2303628847037207
